# jQMM 2 EMU Demonstration Notebook
By Tyler W. H. Backman

This notebook reproduces the first example in Antoniewicz 2006 using the bayflux EMU simulator built on COBRApy and NumPy. 

Inputs, all specified as Python code within this notebook:
* COBRApy model which includes 6 metabolites and 5 reactions
* atom transitions for all 5 reactions listed above
* a flux vector to simulate
* input metabolite labeling (metabolite A)
* an output metabolite to simulate (metabolite F)

Outputs:
* a compiled set of operations to rapidly simulate labeling for any flux vector
* simulated labeling for metabolite F

Notes:
* All functions use numpy matrices or Python lists and tuples, but we display them here as Pandas DataFrames in order to label the rows and columns for illustration purposes.
* EMUs are specified in code with zero-indexed atoms, but are printed as text in output using repr() as 1-indexed, for direct comparison to published literature.

In [1]:
# Note: for use on top of the tbackman/debian-cheminformatics image on dockerhub
# install the development version of jQMM2
# as follows: pip3 install -e ../

In [2]:
import cobra
import bayflux
import pandas as pd
import numpy as np

### First we will use COBRApy to build a model corresponding to the reaction stoichiometry in Fig. 3 of Antoniewicz 2006 which consists of 6 metabolites named A-F, and 5 reactions

<img src="input_data/fig3.png" alt="" style="width: 300px;"/>

In [3]:
# create a blank model
model = cobra.Model('fig3')

# define metabolites
A = cobra.Metabolite(
    'A',
    formula='C3',
    name='A',
    compartment='c')
B = cobra.Metabolite(
    'B',
    formula='C3',
    name='B',
    compartment='c')
C = cobra.Metabolite(
    'C',
    formula='C2',
    name='C',
    compartment='c')
D = cobra.Metabolite(
    'D',
    formula='C3',
    name='D',
    compartment='c')
E = cobra.Metabolite(
    'E',
    formula='C1',
    name='E',
    compartment='c')
F = cobra.Metabolite(
    'F',
    formula='C3',
    name='F',
    compartment='c')

# define reactions and add them to the model
a_b = cobra.Reaction('a_b')
a_b.lower_bound = 0
a_b.upper_bound = 500
a_b.add_metabolites({
    A: -1.0,
    B: 1.0
})
print(a_b.reaction)

b_ec = cobra.Reaction('b_ec')
b_ec.lower_bound = 0
b_ec.upper_bound = 500
b_ec.add_metabolites({
    B: -1.0,
    E: 1.0,
    C: 1.0
})
print(b_ec.reaction)

bc_de = cobra.Reaction('bc_de')
bc_de.lower_bound = 0
bc_de.upper_bound = 500
bc_de.add_metabolites({
    B: -1.0,
    C: -1.0,
    D: 1.0,
    E: 2.0
})
print(bc_de.reaction)

d_f = cobra.Reaction('d_f')
d_f.lower_bound = 0
d_f.upper_bound = 500
d_f.add_metabolites({
    D: -1.0,
    F: 1.0
})
print(d_f.reaction)

b_d = cobra.Reaction('b_d')
b_d.lower_bound = -500
b_d.upper_bound = 500
b_d.add_metabolites({
    B: -1.0,
    D: 1.0
})
print(b_d.reaction)

model.add_reactions([a_b, b_ec, bc_de, d_f, b_d])

A --> B
B --> C + E
B + C --> D + 2.0 E
D --> F
B <=> D


Now we convert this model to a bayflux ReactionNetwork, which inherets 'EnhancedReaction' objects from each cobra.Reaction, allowing us to add in atom transitions

In [4]:
model = bayflux.ReactionNetwork(model)
model

Name,fig3
Memory address,0x04074343550
Number of metabolites,6
Number of reactions,5
Number of groups,0
Objective expression,0
Compartments,c


Here we set the atom transitions for each reaction, and show an example of viewing an EnhancedReaction which reports atom transitions

In [5]:
# create dict of metabolites by name
# we use this instead of directly using the metabolite IDs above, because the conversion to
# a bayflux.ReactionNetwork created new metabolite objects
m = {m.id:m for m in model.metabolites}

model.reactions.a_b.transitions = [bayflux.AtomTransition(
        ((m['A'], [1,2,3]),), # reactant labels
        ((m['B'], [1,2,3]),) # product labels
    )]
model.reactions.b_ec.transitions = [bayflux.AtomTransition(
        ((m['B'], [1,2,3]),), # reactant labels
        ((m['E'], [1]), (m['C'], [2,3]),) # product labels
     )]
model.reactions.bc_de.transitions = [bayflux.AtomTransition(
        ((m['B'], [1,2,3]), (m['C'], [4,5]),), # reactant labels
        ((m['E'], [1]), (m['D'], [2,3,4]), (m['E'], [5]),) # product labels
     )]
model.reactions.d_f.transitions = [bayflux.AtomTransition(
        ((m['D'], [1,2,3]),), # reactant labels
        ((m['F'], [1,2,3]),) # product labels
     )]
model.reactions.b_d.transitions = [bayflux.AtomTransition(
        ((m['B'], [1,2,3]),), # reactant labels
        ((m['D'], [1,2,3]),) # product labels
     )]

model.reactions.b_d

Reaction identifier,b_d
Name,
Memory address,0x0407432ced0
Stoichiometry,B <=> D B <=> D
GPR,
Lower bound,-500
Upper bound,500
Atom transition,B --> D abc : abc


jQMM 1.0 like transition printing

In [6]:
model.reactions.b_d.transitions[0]

B --> D	abc : abc

Here we represent fluxes as a FluxVector object, with fluxes ordered to match the model variables

**Note**: non-reversible reactions should always carry zero reverse flux!

In [7]:
fluxDict = {'a_b': [100, 0], 'b_d': [110, 50], 'b_ec': [20, 0], 'bc_de': [20, 0], 'd_f': [80, 0]}
fluxes = bayflux.FluxVector(model, fluxDict)
fluxes

,Flux
a_b,100.0
a_b_reverse_dbf08,0.0
b_ec,20.0
b_ec_reverse_e30a6,0.0
bc_de,20.0
bc_de_reverse_eb6e8,0.0
d_f,80.0
d_f_reverse_5ee53,0.0
b_d,110.0
b_d_reverse_4731b,50.0


### Perform a test to find all EMU reactions producing metabolite F

In [8]:
F = model.metabolites.get_by_id('F')
outputEMU = bayflux.EMU(F,[0,1,2])
result = bayflux.findProducingEMUTransitions(outputEMU)
result

[1.0*d_f: D[1, 2, 3]-> F[0, 1, 2]]

### perform test full EMU decomposition of metabolite F precursors

In [9]:
fullDecomposition = bayflux.emuDecomposition([outputEMU])
fullDecomposition

[1.0*d_f: D[1, 2, 3]-> F[0, 1, 2],
 1.0*bc_de: B[2, 3], C[1]-> D[1, 2, 3],
 1.0*b_d: B[1, 2, 3]-> D[1, 2, 3],
 1.0*a_b: A[1, 2, 3]-> B[1, 2, 3],
 -1.0*b_d: D[1, 2, 3]-> B[1, 2, 3],
 1.0*b_ec: B[2]-> C[1],
 1.0*a_b: A[2]-> B[2],
 -1.0*b_d: D[2]-> B[2],
 1.0*bc_de: B[3]-> D[2],
 1.0*b_d: B[2]-> D[2],
 1.0*a_b: A[3]-> B[3],
 -1.0*b_d: D[3]-> B[3],
 1.0*bc_de: C[1]-> D[3],
 1.0*b_d: B[3]-> D[3],
 1.0*a_b: A[2, 3]-> B[2, 3],
 -1.0*b_d: D[2, 3]-> B[2, 3],
 1.0*bc_de: B[3], C[1]-> D[2, 3],
 1.0*b_d: B[2, 3]-> D[2, 3]]

### Split apart EMU reactions by product size

In [10]:
transitionsBySize = bayflux.splitBySize(fullDecomposition)
transitionsBySize

{1: [1.0*b_ec: B[2]-> C[1],
  1.0*a_b: A[2]-> B[2],
  -1.0*b_d: D[2]-> B[2],
  1.0*bc_de: B[3]-> D[2],
  1.0*b_d: B[2]-> D[2],
  1.0*a_b: A[3]-> B[3],
  -1.0*b_d: D[3]-> B[3],
  1.0*bc_de: C[1]-> D[3],
  1.0*b_d: B[3]-> D[3]],
 2: [1.0*a_b: A[2, 3]-> B[2, 3],
  -1.0*b_d: D[2, 3]-> B[2, 3],
  1.0*bc_de: B[3], C[1]-> D[2, 3],
  1.0*b_d: B[2, 3]-> D[2, 3]],
 3: [1.0*d_f: D[1, 2, 3]-> F[0, 1, 2],
  1.0*bc_de: B[2, 3], C[1]-> D[1, 2, 3],
  1.0*b_d: B[1, 2, 3]-> D[1, 2, 3],
  1.0*a_b: A[1, 2, 3]-> B[1, 2, 3],
  -1.0*b_d: D[1, 2, 3]-> B[1, 2, 3]]}

###  EMU simulation matrix descriptions

AX=BY
X=A^-1BY

Matrix A: fluxes (internal only, not inputs)
* each column represents fluxes into and out of a given metabolite
* order from top to bottom is fluxes from EMU reactions towards a given other metabolite,
in the same order as X top to bottom
* a metabolite's own cell represents fluxes towards that metabolite from all reactions
* values are fluxes away from the metabolite in that column (e.g. fluxes towards it are negative)

Matrix B (input fluxes only):
* columns are external EMUs from feed
* rows are internal EMUs
* values represent feed fluxes from feed reaction

Matrix Y: input labeling
* each row represents an external EMU with the same order as the columns in Matrix B
* columns represent the number of additional neutrons (0, 1, 2, etc.)
* values represent the mass distribution of each external EMU distributed over the columns

Matrix X: predicted labeling
* each row represents a predicted EMU with the same order as Matrix A
* columns represent the number of additional neutrons (0, 1, 2, etc.)
* values represent the mass distribution fir each predicted EMU

### Test identifying the internal and external EMUs for all size 1 EMU reactions, which is later used as the coordinates for the resulting matrices

In [11]:
matrixCoords = bayflux.constructMatrixCoords(transitionsBySize[1])
matrixCoords

{'internalEMUs': {'text': ['C[1]', 'B[2]', 'D[2]', 'B[3]', 'D[3]'],
  'hashes': [-2382903056126914789,
   -4448711352358535334,
   8823725731396585482,
   -4448711352359700383,
   8823725731395420433],
  'objects': [C[1], B[2], D[2], B[3], D[3]]},
 'externalEMUs': {'text': ['A[2]', 'A[3]'],
  'hashes': [902537528576644536, 902537528575479487],
  'objects': [A[2], A[3]]}}

### Create example zero matrices for the length-1 EMU reactions

In [12]:
exampleMatrices = bayflux.constructZeroMatrices(matrixCoords)
print('Matrix A')
display(pd.DataFrame(exampleMatrices[0], index=matrixCoords['internalEMUs']['text'], columns=matrixCoords['internalEMUs']['text']))
print('Matrix B')
display(pd.DataFrame(exampleMatrices[1], index=matrixCoords['internalEMUs']['text'], columns=matrixCoords['externalEMUs']['text']))

Matrix A


,C[1],B[2],D[2],B[3],D[3]
C[1],0.0,0.0,0.0,0.0,0.0
B[2],0.0,0.0,0.0,0.0,0.0
D[2],0.0,0.0,0.0,0.0,0.0
B[3],0.0,0.0,0.0,0.0,0.0
D[3],0.0,0.0,0.0,0.0,0.0


Matrix B


,A[2],A[3]
C[1],0.0,0.0
B[2],0.0,0.0
D[2],0.0,0.0
B[3],0.0,0.0
D[3],0.0,0.0


### Example of compiling the size 1 EMU reactions into a set of matrix addition operations

In [13]:
operations = bayflux.compileEMUTransitionList(transitionsBySize[1], matrixCoords, model)
pd.DataFrame(operations, columns=('A=0/B=1 matrix selector', 'Row', 'Column', 'Flux selector', 'Factor'))

,A=0/B=1 matrix selector,Row,Column,Flux selector,Factor
0,0.0,0.0,1.0,2.0,1.0
1,0.0,0.0,0.0,2.0,-1.0
2,1.0,1.0,0.0,0.0,-1.0
3,0.0,1.0,1.0,0.0,-1.0
4,0.0,1.0,2.0,9.0,1.0
5,0.0,1.0,1.0,9.0,-1.0
6,0.0,2.0,3.0,4.0,1.0
7,0.0,2.0,2.0,4.0,-1.0
8,0.0,2.0,1.0,8.0,1.0
9,0.0,2.0,2.0,8.0,-1.0


### Now we will compile and execute all A and B matricies for each EMU sub-network size

In this example we make matY semi-manually

#### Size 1 sub-network

In [14]:
# compute A and B matrix for size 1
matrixCoords1 = bayflux.constructMatrixCoords(transitionsBySize[1])
operations1 = bayflux.compileEMUTransitionList(transitionsBySize[1], matrixCoords1, model)
zeroMatrices1 = bayflux.constructZeroMatrices(matrixCoords1)
result1 = bayflux.executeCompiledEMUTransitionList(operations1, zeroMatrices1, fluxes)

print('Matrix A')
display(pd.DataFrame(result1[0], index=matrixCoords1['internalEMUs']['text'], columns=matrixCoords1['internalEMUs']['text']))
print('Matrix B')
display(pd.DataFrame(result1[1], index=matrixCoords1['internalEMUs']['text'], columns=matrixCoords1['externalEMUs']['text']))

Matrix A


,C[1],B[2],D[2],B[3],D[3]
C[1],-20.0,20.0,0.0,0.0,0.0
B[2],0.0,-150.0,50.0,0.0,0.0
D[2],0.0,110.0,-130.0,20.0,0.0
B[3],0.0,0.0,0.0,-150.0,50.0
D[3],20.0,0.0,0.0,110.0,-130.0


Matrix B


,A[2],A[3]
C[1],0.0,0.0
B[2],-100.0,0.0
D[2],0.0,0.0
B[3],0.0,-100.0
D[3],0.0,0.0


In [15]:
# compute input matY for size 1
matY1 = np.matrix([[0, 1], [1, 0]])
pd.DataFrame(matY1, index=matrixCoords1['externalEMUs']['text'])

,0,1
A[2],0,1
A[3],1,0


In [16]:
# compute predicted labeling matX for size 1
matA = result1[0]
matB = result1[1]
matX1 = np.array(np.linalg.inv(matA).dot(matB).dot(matY1))
pd.DataFrame(matX1, index=matrixCoords1['internalEMUs']['text'])

,0,1
C[1],0.066667,0.933333
B[2],0.066667,0.933333
D[2],0.200000,0.800000
B[3],0.933333,0.066667
D[3],0.800000,0.200000


#### Size 2 sub-network

In [17]:
# compute A and B matrix for size 2
matrixCoords2 = bayflux.constructMatrixCoords(transitionsBySize[2])
operations2 = bayflux.compileEMUTransitionList(transitionsBySize[2], matrixCoords2, model)
zeroMatricies2 = bayflux.constructZeroMatrices(matrixCoords2)
result2 = bayflux.executeCompiledEMUTransitionList(operations2, zeroMatricies2, fluxes)

print('Matrix A')
display(pd.DataFrame(result2[0], index=matrixCoords2['internalEMUs']['text'], columns=matrixCoords2['internalEMUs']['text']))
print('Matrix B')
display(pd.DataFrame(result2[1], index=matrixCoords2['internalEMUs']['text'], columns=matrixCoords2['externalEMUs']['text']))

Matrix A


,"B[2, 3]","D[2, 3]"
"B[2, 3]",-150.0,50.0
"D[2, 3]",110.0,-130.0


Matrix B


,"A[2, 3]",B[3] x C[1]
"B[2, 3]",-100.0,0.0
"D[2, 3]",0.0,-20.0


In [18]:
# compute input matY for size 2
# matY2 = pd.DataFrame(0, columns=range(0,3), index=matrixCoords2['externalEMUs']['text'])
matY2 = np.zeros(shape=(len(matrixCoords2['externalEMUs']['text']), 3))
matY2[0] = (0, 1, 0)
matY2[1] = np.convolve(
    matX1[matrixCoords1['internalEMUs']['text'].index('B[3]')].tolist()[0],
    matX1[matrixCoords1['internalEMUs']['text'].index('C[1]')].tolist()[0]
)

pd.DataFrame(matY2, index=matrixCoords2['externalEMUs']['text'])

,0,1,2
"A[2, 3]",0.000000,1.000000,0.000000
B[3] x C[1],0.062222,0.062222,0.062222


In [19]:
# compute predicted labeling matX for size 2
matA = result2[0]
matB = result2[1]
matX2 = np.array(np.linalg.inv(matA).dot(matB).dot(matY2))

pd.DataFrame(matX2, index=matrixCoords2['internalEMUs']['text'])

,0,1,2
"B[2, 3]",0.004444,0.933016,0.004444
"D[2, 3]",0.013333,0.799048,0.013333


#### Size 3 sub-network

In [20]:
# compute A and B matrix for size 3
matrixCoords3 = bayflux.constructMatrixCoords(transitionsBySize[3])
operations3 = bayflux.compileEMUTransitionList(transitionsBySize[3], matrixCoords3, model)
zeroMatricies3 = bayflux.constructZeroMatrices(matrixCoords3)
result3 = bayflux.executeCompiledEMUTransitionList(operations3, zeroMatricies3, fluxes)

print('Matrix A')
display(pd.DataFrame(result3[0], index=matrixCoords3['internalEMUs']['text'], columns=matrixCoords3['internalEMUs']['text']))
print('Matrix B')
display(pd.DataFrame(result3[1], index=matrixCoords3['internalEMUs']['text'], columns=matrixCoords3['externalEMUs']['text']))

Matrix A


,"F[0, 1, 2]","B[1, 2, 3]","D[1, 2, 3]"
"F[0, 1, 2]",-80.0,0.0,80.0
"B[1, 2, 3]",0.0,-150.0,50.0
"D[1, 2, 3]",0.0,110.0,-130.0


Matrix B


,"A[1, 2, 3]","B[2, 3] x C[1]"
"F[0, 1, 2]",0.0,0.0
"B[1, 2, 3]",-100.0,0.0
"D[1, 2, 3]",0.0,-20.0


In [21]:
# compute input matY for size 3
matY3 = np.zeros(shape=(len(matrixCoords3['externalEMUs']['text']), 4))
matY3[0] = (0, 1, 0, 0)
# matY3[1] = np.convolve(matX2.loc['B[2, 3]'], matX1.loc['C[1]'])
matY3[1] = np.convolve(
    matX2[matrixCoords2['internalEMUs']['text'].index('B[2, 3]')].tolist()[0],
    matX1[matrixCoords1['internalEMUs']['text'].index('C[1]')].tolist()[0]
)

pd.DataFrame(matY3, index=matrixCoords3['externalEMUs']['text'])

,0,1,2,3
"A[1, 2, 3]",0.000000,1.000000,0.000000,0.000000
"B[2, 3] x C[1]",0.000296,0.000296,0.000296,0.000296


In [22]:
# compute predicted labeling matX for size 3
matA = result3[0]
matB = result3[1]
matX3 = np.array(np.linalg.inv(matA).dot(matB).dot(matY3))

pd.DataFrame(matX3, index=matrixCoords3['internalEMUs']['text'])

,0,1,2,3
"F[0, 1, 2]",0.000063,0.785778,0.000063,0.000063
"B[1, 2, 3]",0.000021,0.928593,0.000021,0.000021
"D[1, 2, 3]",0.000063,0.785778,0.000063,0.000063


### Now we define the substrate labeling vector, and test the ability to extract a mass distribution vector from this labeling

In [23]:
# demo creation of a mass distribution vector for a binary labeling vector

substrateLabelingDict = {
    m['A']: ((1.0, [0, 1, 0]),)
}

inputEMU = bayflux.EMU(m['A'], [0,1,2])
extractedLabeling = bayflux.extractSubstrateEMU(inputEMU, substrateLabelingDict)
extractedLabeling

array([0., 1., 0., 0.])

### Run fully automated EMU simulation

#### First let's decompose the EMU network, and compile its structure into a list of matrix addition operations

In [24]:
# EMUs to simulate in list form
emusToSimulate = [bayflux.EMU(m['F'],[0,1,2])]

compiledData = bayflux.emuCompile(emusToSimulate, model, substrateLabelingDict, True, True)
compiledData

[{'size': 1,
  'matrixCoords': {'internalEMUs': {'text': ['C[1]', 'D[2]', 'B[3]', 'D[3]'],
    'hashes': [-2382903056126914789,
     8823725731396585482,
     -4448711352359700383,
     8823725731395420433],
    'objects': [C[1], D[2], B[3], D[3]]},
   'externalEMUs': {'text': ['A[2]', 'A[3]'],
    'hashes': [902537528576644536, 902537528575479487],
    'objects': [A[2], A[3]]}},
  'operations': array([[ 1.,  0.,  0.,  0., -1.],
         [ 0.,  0.,  0.,  0., -1.],
         [ 0.,  0.,  1.,  9.,  1.],
         [ 0.,  0.,  0.,  9., -1.],
         [ 0.,  1.,  2.,  4.,  1.],
         [ 0.,  1.,  1.,  4., -1.],
         [ 0.,  1.,  0.,  8.,  1.],
         [ 0.,  1.,  1.,  8., -1.],
         [ 1.,  2.,  1.,  0., -1.],
         [ 0.,  2.,  2.,  0., -1.],
         [ 0.,  2.,  3.,  9.,  1.],
         [ 0.,  2.,  2.,  9., -1.],
         [ 0.,  3.,  0.,  4.,  1.],
         [ 0.,  3.,  3.,  4., -1.],
         [ 0.,  3.,  2.,  8.,  1.],
         [ 0.,  3.,  3.,  8., -1.]]),
  'substrateMDVs': [array

#### test automated creation of a matrix Y

In [25]:
results = {1: matX1, 2: matX2, 3: matX3}
substrateMDVs, operationsY = bayflux.compileMatrixY(compiledData, compiledData[2]['matrixCoords'], 3, substrateLabelingDict)
autoMatY3 = bayflux.executeCompiledMatrixY(substrateMDVs, operationsY, results, 3)
pd.DataFrame(autoMatY3, index=compiledData[2]['matrixCoords']['externalEMUs']['text'])

,0,1,2,3
"A[1, 2, 3]",0.000000,1.000000,0.000000,0.000000
"B[2, 3] x C[1]",0.000296,0.066349,0.871111,0.004148


#### Now we'll test the injection of a specific flux vector, in order to simulate the labeling of metabolite F

In [26]:
fluxes

,Flux
a_b,100.0
a_b_reverse_dbf08,0.0
b_ec,20.0
b_ec_reverse_e30a6,0.0
bc_de,20.0
bc_de_reverse_eb6e8,0.0
d_f,80.0
d_f_reverse_5ee53,0.0
b_d,110.0
b_d_reverse_4731b,50.0


In [27]:
results = bayflux.simulateLabeling(compiledData, fluxes, substrateLabelingDict)
results

{1: array([[0.06666667, 0.93333333],
        [0.2       , 0.8       ],
        [0.93333333, 0.06666667],
        [0.8       , 0.2       ]]),
 2: array([[0.00444444, 0.99111111, 0.00444444],
        [0.01333333, 0.97333333, 0.01333333]]),
 3: array([[6.34920635e-05, 8.00761905e-01, 1.98285714e-01, 8.88888889e-04],
        [2.11640212e-05, 9.33587302e-01, 6.60952381e-02, 2.96296296e-04]])}

#### Now view final results and compare to published metabolite F MDV of [0.0001, 0.8008, 0.1983, 0.0009]

In [28]:
pd.DataFrame(results[3], index=compiledData[2]['matrixCoords']['internalEMUs']['text'])

,0,1,2,3
"F[0, 1, 2]",0.000063,0.800762,0.198286,0.000889
"B[1, 2, 3]",0.000021,0.933587,0.066095,0.000296
